In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Importing Deep Learning Libraries
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

# Displaying Images

In [ ]:
# taking inputs images for training
picture_size = 48
folder_path = "../input/face-expression-recognition-dataset/images/"

In [ ]:
# checking image
expression = 'fear'

plt.figure(figsize= (12,12))
for i in range(1,10,1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"train/"+expression+"/"+os.listdir(folder_path + "train/" + expression)[i], target_size=(picture_size,picture_size))
    plt.imshow(img)
plt.show()

###### Making Training and Validation Data

In [ ]:
# splitting data into train, test and validation set
batch_size = 32

datagen_train = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+"train", target_size = (picture_size,
                                                                                  picture_size),
                                             color_mode = "grayscale", batch_size = batch_size, 
                                              class_mode = "categorical", shuffle=True)

test_set = datagen_val.flow_from_directory(folder_path+"validation", target_size = (picture_size,
                                                                                    picture_size),
                                          color_mode = "grayscale", batch_size = batch_size, 
                                              class_mode = "categorical", shuffle=True)

# Model Building

In [ ]:
# building model with 7 classes
from tensorflow.keras.optimizers import Adam,SGD,RMSprop


no_of_classes = 7

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))



opt = Adam(learning_rate = 0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Visualize Model

In [ ]:
# visualizing the model
import tensorflow as tf
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
)

# Fitting the Model with Training and Validation Data

In [ ]:
# importing tensorflow library and package
from tensorflow.keras.optimizers import RMSprop,Adam,SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath="./model.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True,mode='max')
#Stopping training when a monitored metric has stopped improving.
early_stopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=0.00005,
                              patience=11,
                              verbose=1,
                              restore_best_weights=True
                              )

reduce_learningrate = ReduceLROnPlateau(monitor='val_accuracy',
                                       factor=0.5,
                                       patience=7,
                                       verbose=1,
                                       min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 50

In [ ]:
history = model.fit_generator(generator=train_set,
                             steps_per_epoch=train_set.n//train_set.batch_size,
                             epochs=epochs,validation_data=test_set,validation_steps=test_set.n//test_set.batch_size,
                             callbacks=callbacks_list
                             )

# Plotting Accuracy & Loss

In [ ]:
# plotting graph to check accuracy and loss

plt.figure(figsize=(8,8))
x1 = [1,2,3,4,5,6,7,8,9,10]
y1 = [0.3428,0.4753,0.5258,0.566,0.5761,0.5939,0.6149,0.6327,0.6514,0.6742]
plt.plot(x1, y1, label = "training")

x2 = [1,2,3,4,5,6,7,8,9,10]
y2 = [0.3631,0.4966,0.4251,0.5031,0.5626,0.5888,0.592,0.5256,0.6058,0.6027]
plt.plot(x2, y2, label = "validation")


plt.xlabel('epoch')
plt.title('accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# plotting graph to check accuracy and loss
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.suptitle('Optimizer : Adam',fontsize=10)
plt.ylabel('Loss',fontsize=16)
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1,2,2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
df = pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')
print(df.shape)
df.head()

In [ ]:
df.emotion.unique()

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
df.emotion.value_counts()

In [ ]:
df.shape

In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)
img_array.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
import scikitplot
from sklearn.metrics import classification_report

yhat_valid = model.predict(X_valid)
yhat_valid = np.argmax(yhat_valid, axis=1)
scikitplot.metrics.plot_confusion_matrix(np.argmax(y_valid, axis=1), yhat_valid, figsize=(7,7))
plt.savefig("confusion_matrix.png")

test_accu = np.sum(np.argmax(y_valid, axis=1) == yhat_valid) / len(np.argmax(y_valid, axis=1) == yhat_valid) * 100
print(f'test accuracy: {round(test_accu, 4)} %\n\n')
print(classification_report(np.argmax(y_valid, axis=1), yhat_valid))

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

# Normalizing results, as neural networks are very sensitive to unnormalized data.
X_train = X_train / 255.
X_valid = X_valid / 255.